# Classifying cars

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import glob
import shutil
import itertools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG19, MobileNet, InceptionV3, EfficientNetB6

# Creating directories

In [ ]:
os.mkdir('/kaggle/working/train')
os.mkdir('/kaggle/working/test')

path = '../input/100-images-of-top-50-car-brands/imgs_zip/imgs/'
for class_ in os.listdir(path)[:20]:
    os.mkdir('/kaggle/working/train/'+class_)
    os.mkdir('/kaggle/working/test/'+class_)
    
    data = os.listdir(path+class_)
    for img in itertools.islice(glob.iglob(os.path.join(path+class_, '*jpg')), 
                                 len(data[:round(len(data)*0.8)])):
        shutil.copy(img, '/kaggle/working/train/'+class_)
        
    for img in itertools.islice(glob.iglob(os.path.join(path+class_, '*jpg')), 
                                 len(data[:round(len(data)*0.8)]), len(data)):
        shutil.copy(img, '/kaggle/working/test/'+class_)

# Augmenting images

In [ ]:
train_gen = ImageDataGenerator().flow_from_directory('/kaggle/working/train',
                                              target_size=(128, 128), class_mode='categorical')
test_gen = ImageDataGenerator().flow_from_directory('/kaggle/working/test',
                                              target_size=(128, 128), class_mode='categorical')

# Training models

In [ ]:
names = ['MobileNet', 'InceptionV3', 'EfficientNetB6']
histories = []

for functional in [MobileNet(include_top=False, input_shape=(128, 128, 3)),
                   InceptionV3(include_top=False, input_shape=(128, 128, 3)),
                   EfficientNetB6(include_top=False, input_shape=(128, 128, 3))]:
    model = Sequential()
    model.add(functional)
    model.add(Flatten())
    model.add(Dense(20, activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics='accuracy', optimizer='adam')
    histories.append(model.fit_generator(train_gen, validation_data=test_gen, 
                                         steps_per_epoch=10, validation_steps=10, epochs=10))

# Evaluating performance

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for metric in histories[0].history:
    i = 0
    ax = axes.flatten()[list(histories[0].history).index(metric)]
    for history in histories:
        ax.set_title(metric + " for different models")
        ax.plot(history.history[metric], label=names[i])
        i += 1
    ax.legend()

plt.show()

## Thank you for reading my notebook.

## If you enjoyed this notebook and found it helpful, please give it an upvote and provide feedback as it would help me make more of these.